In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.preprocessing import image_dataset_from_directory
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# --- (Optionnel) précision mixte pour GPU ---
# tf.keras.mixed_precision.set_global_policy('mixed_float16')

# 1) TPU/GPU
try:
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    strategy = tf.distribute.TPUStrategy(resolver)
    print("TPU activé")
except:
    strategy = tf.distribute.MirroredStrategy()
    print("GPU/CPU activé")
print("Répliques :", strategy.num_replicas_in_sync)

E0000 00:00:1745614756.265469      10 common_lib.cc:612] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: ===
learning/45eac/tfrc/runtime/common_lib.cc:230


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


I0000 00:00:1745614780.224299      10 service.cc:148] XLA service 0x5a0e515462f0 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745614780.224348      10 service.cc:156]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1745614780.224352      10 service.cc:156]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1745614780.224355      10 service.cc:156]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1745614780.224358      10 service.cc:156]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1745614780.224360      10 service.cc:156]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1745614780.224362      10 service.cc:156]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1745614780.224383      10 service.cc:156]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1745614780.224386      10 service.cc:156]   StreamExecutor device (7): TPU, 2a886c8


GPU/CPU activé
Répliques : 1


In [2]:
# 2) Hyperparamètres allégés
IMG_SIZE   = (256, 256)
BATCH_SIZE = 32    #<<< batch réduit
SEED       = 42
EPOCH_HEAD = 18
EPOCH_WARM = 12
EPOCH_FINE = 16

PATHS = {
    "PV":  "/kaggle/input/plantdisease/PlantVillage",
    "PLD_T": "/kaggle/input/potato-disease-leaf-datasetpld/PLD_3_Classes_256/Training",
    "PLD_E": "/kaggle/input/potato-disease-leaf-datasetpld/PLD_3_Classes_256/Testing"
}

CLASS_NAMES = [
    "Pepper__bell___Bacterial_spot","Pepper__bell___healthy",
    "Potato___Early_blight","Potato___Late_blight","Potato___healthy",
    "Tomato_Bacterial_spot","Tomato_Early_blight","Tomato_Late_blight",
    "Tomato_Leaf_Mold","Tomato_Septoria_leaf_spot",
    "Tomato_Spider_mites_Two_spotted_spider_mite","Tomato__Target_Spot",
    "Tomato__Tomato_YellowLeaf__Curl_Virus","Tomato__Tomato_mosaic_virus",
    "Tomato_healthy"
]
NUM_CLASSES = len(CLASS_NAMES)
AUTOTUNE    = tf.data.AUTOTUNE


In [3]:
# 3) Load PlantVillage
def load_pv():
    ds_tr = image_dataset_from_directory(
        PATHS["PV"], validation_split=0.2, subset="training",
        seed=SEED, image_size=IMG_SIZE, batch_size=BATCH_SIZE,
        label_mode="int"
    )
    ds_val_full = image_dataset_from_directory(
        PATHS["PV"], validation_split=0.2, subset="validation",
        seed=SEED, image_size=IMG_SIZE, batch_size=BATCH_SIZE,
        label_mode="int"
    )
    n = tf.data.experimental.cardinality(ds_val_full).numpy()
    return ds_tr, ds_val_full.skip(n//2), ds_val_full.take(n//2)

In [4]:
# 4) Load PLD & remap
def load_pld():
    names = ["Early_Blight","Late_Blight","Healthy"]
    ds_tr = image_dataset_from_directory(
        PATHS["PLD_T"], shuffle=True, seed=SEED,
        image_size=IMG_SIZE, batch_size=BATCH_SIZE,
        label_mode="int", class_names=names
    )
    ds_te = image_dataset_from_directory(
        PATHS["PLD_E"], shuffle=False,
        image_size=IMG_SIZE, batch_size=BATCH_SIZE,
        label_mode="int", class_names=names
    )
    mapping = tf.constant([
        CLASS_NAMES.index("Potato___Early_blight"),
        CLASS_NAMES.index("Potato___Late_blight"),
        CLASS_NAMES.index("Potato___healthy")
    ], dtype=tf.int32)
    ds_tr = ds_tr.map(lambda x,y:(x,tf.gather(mapping,y)), AUTOTUNE)
    ds_te = ds_te.map(lambda x,y:(x,tf.gather(mapping,y)), AUTOTUNE)
    return ds_tr, ds_te

pv_tr_raw, pv_val_raw, pv_te_raw = load_pv()
pld_tr_raw, pld_te_raw          = load_pld()

Found 20638 files belonging to 15 classes.
Using 16511 files for training.
Found 20638 files belonging to 15 classes.
Using 4127 files for validation.
Found 3251 files belonging to 3 classes.
Found 405 files belonging to 3 classes.


In [5]:
# 5) One-hot + preprocess
def preprocess_onehot(x,y):
    x = preprocess_input(tf.cast(x,tf.float32))
    y = tf.one_hot(y, NUM_CLASSES)
    return x,y

pv_train = pv_tr_raw.map(preprocess_onehot, AUTOTUNE)
pv_val   = pv_val_raw.map(preprocess_onehot, AUTOTUNE)
pv_test  = pv_te_raw.map(preprocess_onehot, AUTOTUNE)
pld_train= pld_tr_raw.map(preprocess_onehot, AUTOTUNE)
pld_test = pld_te_raw.map(preprocess_onehot, AUTOTUNE)

# 6) Combine + augmentation Keras
train_ds = pv_train.concatenate(pld_train)
val_ds   = pv_val
test_ds  = pv_test.concatenate(pld_test)

augment = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])
train_ds = (train_ds
    .shuffle(1000, seed=SEED)
    .map(lambda x,y:(augment(x,training=True),y), AUTOTUNE)
    # >>> plus de cache() ici pour économiser la RAM
    .prefetch(AUTOTUNE)
)
val_ds  = val_ds.prefetch(AUTOTUNE)
test_ds = test_ds.prefetch(AUTOTUNE)

In [6]:
# 7) Class weights depuis train_ds
all_labels = np.concatenate([np.argmax(y,axis=1)
    for _,y in train_ds.unbatch().batch(10000)], axis=0)
cw = compute_class_weight("balanced",
                          classes=np.arange(NUM_CLASSES),
                          y=all_labels)
class_weights = dict(enumerate(cw))
print("Class weights:", class_weights)

Class weights: {0: np.float64(1.6530322040987036), 1: np.float64(1.120294784580499), 2: np.float64(0.6232103437401451), 3: np.float64(0.6777091906721536), 4: np.float64(1.4060476698683742), 5: np.float64(0.781415579280348), 6: np.float64(1.6285125669550886), 7: np.float64(0.8402210884353741), 8: np.float64(1.7176879617557583), 9: np.float64(0.9553782934493594), 10: np.float64(0.9802579365079365), 11: np.float64(1.152639253426655), 12: np.float64(0.5118363118363118), 13: np.float64(4.465988700564972), 14: np.float64(1.0719826417141307)}


In [7]:
# 8) Build & compile
with strategy.scope():
    base = EfficientNetB0(include_top=False, weights="imagenet",
                          input_shape=(*IMG_SIZE,3))
    base.trainable = False

    inp = layers.Input(shape=(*IMG_SIZE,3))
    x   = augment(inp)
    x   = preprocess_input(x)
    x   = base(x, training=False)
    x   = layers.GlobalAveragePooling2D()(x)
    x   = layers.Dropout(0.3)(x)
    out = layers.Dense(NUM_CLASSES, activation="softmax")(x)

    model = models.Model(inp,out)
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])
model.summary()

cb1 = [
    EarlyStopping("val_accuracy",patience=3,restore_best_weights=True),
    ModelCheckpoint("head.h5","val_accuracy",save_best_only=True),
    ReduceLROnPlateau("val_loss",factor=0.5,patience=2,verbose=1)
]
cb2 = [
    EarlyStopping("val_accuracy",patience=3,restore_best_weights=True),
    ModelCheckpoint("ft.h5","val_accuracy",save_best_only=True),
    ReduceLROnPlateau("val_loss",factor=0.5,patience=2,verbose=1)
]

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 8, 8, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 15)             │        19,215 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,068,786 (15.52 MB)

 Trainable params: 19,215 (75.06 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [ ]:
# 9) Phase 1 : tête
with strategy.scope():
    h1 = model.fit(train_ds, validation_data=val_ds,
                   epochs=EPOCH_HEAD,
                   class_weight=class_weights, callbacks=cb1)
model.load_weights("head.h5")

# 10) Phase 2 : warm-up
with strategy.scope():
    for l in base.layers[:-20]: l.trainable=False
    for l in base.layers[-20:]:
        if not isinstance(l,layers.BatchNormalization): l.trainable=True
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])
    h2 = model.fit(train_ds, validation_data=val_ds,
                   epochs=EPOCH_WARM,
                   class_weight=class_weights, callbacks=cb1)

Epoch 1/18
618/618 ━━━━━━━━━━━━━━━━━━━━ 0s 534ms/step - accuracy: 0.5948 - loss: 1.4765

618/618 ━━━━━━━━━━━━━━━━━━━━ 379s 597ms/step - accuracy: 0.5951 - loss: 1.4757 - val_accuracy: 0.8110 - val_loss: 0.6170 - learning_rate: 0.0010
Epoch 2/18
618/618 ━━━━━━━━━━━━━━━━━━━━ 0s 535ms/step - accuracy: 0.8564 - loss: 0.5131

618/618 ━━━━━━━━━━━━━━━━━━━━ 368s 590ms/step - accuracy: 0.8564 - loss: 0.5130 - val_accuracy: 0.8398 - val_loss: 0.4993 - learning_rate: 0.0010
Epoch 3/18
618/618 ━━━━━━━━━━━━━━━━━━━━ 0s 534ms/step - accuracy: 0.8767 - loss: 0.4212

618/618 ━━━━━━━━━━━━━━━━━━━━ 366s 590ms/step - accuracy: 0.8767 - loss: 0.4212 - val_accuracy: 0.8461 - val_loss: 0.4491 - learning_rate: 0.0010
Epoch 4/18
618/618 ━━━━━━━━━━━━━━━━━━━━ 0s 534ms/step - accuracy: 0.8921 - loss: 0.3681

618/618 ━━━━━━━━━━━━━━━━━━━━ 367s 590ms/step - accuracy: 0.8921 - loss: 0.3680 - val_accuracy: 0.8543 - val_loss: 0.4152 - learning_rate: 0.0010
Epoch 5/18
618/618 ━━━━━━━━━━━━━━━━━━━━ 0s 537ms/step - accuracy: 0.8991 - loss: 0.3393

618/618 ━━━━━━━━━━━━━━━━━━━━ 371s 598ms/step - accuracy: 0.8991 - loss: 0.3393 - val_accuracy: 0.8826 - val_loss: 0.3365 - learning_rate: 0.0010
Epoch 6/18
618/618 ━━━━━━━━━━━━━━━━━━━━ 364s 585ms/step - accuracy: 0.9131 - loss: 0.2972 - val_accuracy: 0.8716 - val_loss: 0.3650 - learning_rate: 0.0010
Epoch 7/18
618/618 ━━━━━━━━━━━━━━━━━━━━ 0s 529ms/step - accuracy: 0.9135 - loss: 0.2812
Epoch 7: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
618/618 ━━━━━━━━━━━━━━━━━━━━ 362s 583ms/step - accuracy: 0.9135 - loss: 0.2812 - val_accuracy: 0.8672 - val_loss: 0.3722 - learning_rate: 0.0010
Epoch 8/18
618/618 ━━━━━━━━━━━━━━━━━━━━ 364s 586ms/step - accuracy: 0.9190 - loss: 0.2737 - val_accuracy: 0.8822 - val_loss: 0.3402 - learning_rate: 5.0000e-04
Epoch 1/12
618/618 ━━━━━━━━━━━━━━━━━━━━ 0s 556ms/step - accuracy: 0.9159 - loss: 0.2674

618/618 ━━━━━━━━━━━━━━━━━━━━ 394s 622ms/step - accuracy: 0.9159 - loss: 0.2674 - val_accuracy: 0.9245 - val_loss: 0.2317 - learning_rate: 1.0000e-04
Epoch 2/12
618/618 ━━━━━━━━━━━━━━━━━━━━ 384s 618ms/step - accuracy: 0.9508 - loss: 0.1545 - val_accuracy: 0.8956 - val_loss: 0.3060 - learning_rate: 1.0000e-04
Epoch 3/12
618/618 ━━━━━━━━━━━━━━━━━━━━ 382s 615ms/step - accuracy: 0.9613 - loss: 0.1141 - val_accuracy: 0.9240 - val_loss: 0.2258 - learning_rate: 1.0000e-04
Epoch 4/12
618/618 ━━━━━━━━━━━━━━━━━━━━ 0s 559ms/step - accuracy: 0.9667 - loss: 0.0933

618/618 ━━━━━━━━━━━━━━━━━━━━ 386s 621ms/step - accuracy: 0.9667 - loss: 0.0932 - val_accuracy: 0.9553 - val_loss: 0.1340 - learning_rate: 1.0000e-04
Epoch 5/12
323/618 ━━━━━━━━━━━━━━━━━━━━ 2:44 558ms/step - accuracy: 0.9725 - loss: 0.0820

In [ ]:
# 11) Phase 3 : fine-tuning complet
with strategy.scope():
    base.trainable = True
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])
    h3 = model.fit(train_ds, validation_data=val_ds,
                   epochs=EPOCH_FINE,
                   class_weight=class_weights, callbacks=cb2)
model.load_weights("ft.h5")

# 12) Évaluation
loss,acc = model.evaluate(test_ds,verbose=0)
print(f"Test Acc finale : {acc*100:.2f}%")
y_true = np.concatenate([np.argmax(y,1) for _,y in test_ds],axis=0)
y_pred = np.argmax(model.predict(test_ds),axis=1)
print(classification_report(y_true,y_pred,target_names=CLASS_NAMES))

In [ ]:
# 13) Visualisation
epochs = list(range(1, EPOCH_HEAD+EPOCH_WARM+EPOCH_FINE+1))
acc_t = h1.history["accuracy"] + h2.history["accuracy"] + h3.history["accuracy"]
acc_v = h1.history["val_accuracy"] + h2.history["val_accuracy"] + h3.history["val_accuracy"]
loss_t= h1.history["loss"]     + h2.history["loss"]     + h3.history["loss"]
loss_v= h1.history["val_loss"] + h2.history["val_loss"] + h3.history["val_loss"]

plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(epochs,acc_t,label="Train Acc")
plt.plot(epochs,acc_v,label="Val Acc")
plt.title("Accuracy"); plt.legend()
plt.subplot(1,2,2)
plt.plot(epochs,loss_t,label="Train Loss")
plt.plot(epochs,loss_v,label="Val Loss")
plt.title("Loss"); plt.legend()
plt.tight_layout(); plt.show()

cm = confusion_matrix(y_true,y_pred)
plt.figure(figsize=(12,10))
sns.heatmap(cm,annot=True,fmt="d",cmap="Blues",
            xticklabels=CLASS_NAMES,yticklabels=CLASS_NAMES)
plt.xlabel("Prédit"); plt.ylabel("Vrai"); plt.title("Confusion"); plt.show()

In [ ]:
# 14) Sauvegarde
model.save("pest_final.keras")
open("pest_final.tflite","wb").write(
    tf.lite.TFLiteConverter.from_keras_model(model).convert()
)
print("Terminé : modèles sauvés.")